In [37]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint

# Import API key
import gmaps
import gmaps.datasets
import os

from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
cities_weather_df=pd.read_csv("output_data/cities_weather.csv")
cities_weather_df.dtypes
cities_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Atuona,-9.8000,-139.0333,80.94,77,4,11.36,PF,1616124211
1,Dingle,10.9995,122.6711,86.47,66,62,8.59,PH,1616124422
2,Vadsø,70.0744,29.7487,3.20,92,82,6.91,NO,1616124424
3,Provideniya,64.3833,-173.3000,1.40,60,0,2.24,RU,1616124426
4,Bredasdorp,-34.5322,20.0403,60.80,94,100,6.91,ZA,1616124381


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Use the Lat and Lng as locations
locations = cities_weather_df[["Lat", "Lng"]]
locations

# and Humidity as the weight.
humidity = cities_weather_df["Humidity"]
humidity

# Add Heatmap layer to map
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)

Figure = fig

In [15]:
# Customize the size of the figure *Running Gmaps*

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
# Narrow down the cities to fit weather conditions

# ultilise .loc and use a list of boolean to find max temperature lower than 80 degrees but higher than 70
# Wind speed less than 10 mph.
# Zero cloudiness.

# Note filtering with .loc:(https://towardsdatascience.com/effective-data-filtering-in-pandas-using-loc-40eb815455b6)
desired_cities = cities_weather_df.loc[(cities_weather_df['Max Temp'] < 80) & (cities_weather_df['Max Temp'] > 70)
                                      & (cities_weather_df['Wind Speed'] < 10) 
                                       & (cities_weather_df['Cloudiness'] == 0),:]

#.dropna() to remove the cities that does  not meet the criterias
desired_cities = desired_cities.dropna(how='any')

# fix index
desired_cities.reset_index(inplace=True)
del desired_cities['index']

#flatten the index
desired_cities.columns = [''.join(col).strip() for col in desired_cities.columns.values] 

desired_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Sur,22.5667,59.5289,79.74,39,0,4.21,OM,1616124471
1,Esperance,-33.8667,121.9000,78.80,38,0,8.05,AU,1616124404
2,Codrington,-38.2667,141.9667,74.23,56,0,9.31,AU,1616124419
3,Natal,-5.7950,-35.2094,78.80,83,0,6.91,BR,1616124246
4,New Norfolk,-42.7826,147.0587,75.99,37,0,1.01,AU,1616124715
5,Abu Dhabi,24.4667,54.3667,78.80,22,0,9.22,AE,1616124648
6,Beloha,-25.1667,45.0500,71.92,86,0,4.27,MG,1616124525
7,Taoudenni,22.6783,-3.9836,70.11,16,0,7.23,ML,1616124343
8,Bonthe,7.5264,-12.5050,76.82,80,0,3.89,SL,1616124816
9,Ruwi,23.5996,58.5507,75.20,31,0,2.30,OM,1616124824


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
#Store into variable named `hotel_df`
#hotels_df = pd.Dataframe(desired_cities)
hotel_df = desired_cities
# Add a "Hotel Name" column to the DataFrame
# Get address and Hotel Ratings
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""
hotel_df["Hotel Rating"] = ""

hotel_df.dtypes
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address,Hotel Rating
0,Sur,22.5667,59.5289,79.74,39,0,4.21,OM,1616124471,,,
1,Esperance,-33.8667,121.9000,78.80,38,0,8.05,AU,1616124404,,,
2,Codrington,-38.2667,141.9667,74.23,56,0,9.31,AU,1616124419,,,
3,Natal,-5.7950,-35.2094,78.80,83,0,6.91,BR,1616124246,,,
4,New Norfolk,-42.7826,147.0587,75.99,37,0,1.01,AU,1616124715,,,


In [18]:
# set up a parameters dictionary ,"keyword": "hotel"
# Set parameters to search for hotels within 5 km of coordinates
    # change location each iteration while leaving original params in place
    #"location": location,

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    
    try:
        
    # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
        params = {
        "radius": 5000,
        "type": "lodging",
        "keyword": "hotel",
        "key": g_key
        }

    # get lat, lng from df
        lat = row["Lat"]
        lng = row["Lng"]
    
    # location params *critical* for API
        params["location"] = f"{lat},{lng}"

    # run a request using our params
        hotel_response = requests.get(base_url, params=params)

    # convert to json
        hotel_response = hotel_response.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
        hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = hotel_response["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = hotel_response["results"][0]["rating"]
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        #hotel_df.loc[index, "Hotel Name"] = "NaN"
    continue

Missing field/result... skipping.
Missing field/result... skipping.


In [19]:
pprint(hotel_response)

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -25.1749771, 'lng': 46.0974991},
                           'viewport': {'northeast': {'lat': -25.17366807010728,
                                                      'lng': 46.09893712989273},
                                        'southwest': {'lat': -25.17636772989272,
                                                      'lng': 46.09623747010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Hôtel Étoile du Sud',
              'place_id': 'ChIJVb9WimAD1CERSMzyiuG6yY8',
              'plus_code': {'compound_code': 'R3GW+2X Ambovombe, Madagascar',
                            'global_code': '5H68R3GW+2X'},
              'rating': 0,
              'reference': 'ChIJVb9WimAD1CERSMzyiuG6yY8',
              'scope': 'GOOGLE',
              'types': ['lodging', 'point_of_interest', 'establ

In [20]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address,Hotel Rating
0,Sur,22.5667,59.5289,79.74,39,0,4.21,OM,1616124471,Sur Plaza Hotel,Sur,3.8
1,Esperance,-33.8667,121.9000,78.80,38,0,8.05,AU,1616124404,Esperance Chalet Village,"LOT 49 Frank Freeman Dr, Esperance",4.8
2,Codrington,-38.2667,141.9667,74.23,56,0,9.31,AU,1616124419,Codrington Gardens Bed And Breakfast,"4887 Princes Hwy, Yambuk",4.6
3,Natal,-5.7950,-35.2094,78.80,83,0,6.91,BR,1616124246,Hotel Senac Barreira Roxa,"Via Costeira Sen. Dinarte Medeiros Mariz, 4020...",4.8
4,New Norfolk,-42.7826,147.0587,75.99,37,0,1.01,AU,1616124715,The Woodbridge,"6 Bridge St, New Norfolk",4.8
5,Abu Dhabi,24.4667,54.3667,78.80,22,0,9.22,AE,1616124648,Emirates Palace,W Corniche Rd,4.8
6,Beloha,-25.1667,45.0500,71.92,86,0,4.27,MG,1616124525,,,
7,Taoudenni,22.6783,-3.9836,70.11,16,0,7.23,ML,1616124343,,,
8,Bonthe,7.5264,-12.5050,76.82,80,0,3.89,SL,1616124816,Bonthe Holiday Village,Bonthe,4.5
9,Ruwi,23.5996,58.5507,75.20,31,0,2.30,OM,1616124824,Naseem Hotel,Muscat,3.6


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))

In [23]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '1100px',
    'height': '900px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Display figure
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…